# Optimization Of Algo Case 1

- Maximize Total Return from 2015-Now
- Minimize Sharpe Ratio: Avg Daily Return - Risk Free Rate(0.05) / STDEV(Daily Return)

- Paramters: 
    - Number of Stocks(NumStocks) int between (2,30)
    - How long to hold the stock(holdTime) int between (0,2)
    - Stop Loss float between (-0.01,-0.05)
    - How long to calculate the Percent Change from int between(2,7)

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pandas_market_calendars as mcal
import datetime
from Tools.Trade_Functions import *
import numpy as np
import mlflow
import optuna

/Users/atulkrishnan/Desktop/TradingAlgos/Stock_Picker_Algo


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
experiment_counter=0

In [ ]:
mlflow.set_experiment("Momentum Rider Tuning:V3")

In [ ]:
TimeInput = 2
NumStocks=15
holdTime=0
stopLoss=-0.01
cal=get_calendar()
#Date Range
DateRange=['2015-01-02','2024-11-07']
DateRange=[cal.index(DateRange[0]),cal.index(DateRange[1])]
TotalReturnList = list()



#Run Algorithim
for day in cal[DateRange[0]:DateRange[1]]:
    dailyReturn = pick_trade(RunDate=day,NumStocks=NumStocks,TimeInput=TimeInput,stopLoss=stopLoss,holdTime=holdTime,debug=False)
    TotalReturnList.append(dailyReturn)
returnDf = pd.DataFrame({'Date': pd.to_datetime(cal[DateRange[0]:DateRange[1]]),'Return':TotalReturnList})
returnDf['Date'] = pd.to_datetime(returnDf['Date'])

#Compound Retruns
M = 1
returnVal=[]
for i in returnDf["Return"]:
    M = M*(1+i)
    returnVal.append(M)
returnDf=pd.concat([returnDf,pd.DataFrame({"ReturnVals":returnVal})],axis=1)

#Calculate Metrics
SharpeRatio =(np.mean(returnDf["Return"]) * 252 - 0.05) / (np.std(returnDf["Return"]) * np.sqrt(252))
CumulativeReturn = returnDf.loc[len(returnDf)-1,"ReturnVals"]
max_drawdown = calculate_mdd(returnDf["Return"])
sortino_ratio = calculate_sortino_ratio(returnDf["Return"])


pLratio = len(returnDf["Return"][returnDf["Return"]>0])/ len(returnDf["Return"])

avg_loss= np.mean(returnDf["Return"][returnDf["Return"]<0])
avg_prof= np.mean(returnDf["Return"][returnDf["Return"]>0])



#Saving A plot
plt.figure(figsize=(15, 6))  # Set the figure size
plt.plot(returnDf['Date'], returnDf['ReturnVals'],  linestyle='-', color='blue', label="Return Vals")
plt.title("Return Values Over Time", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Return Vals", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12)
plt.savefig("Visulizations/returns.png", dpi=300, bbox_inches='tight')
plt.close()
#LOG Paramters, Metrics and Artifacts





yearly_plots(returnDf)


for year in range(2015,2025):
    returnDf_year = returnDf[returnDf['Date'].dt.year == year]
    returnDf_year['Return'] = returnDf_year['Return'].astype(float)
   





In [ ]:
for year in range(2015,2025):
    returnDf_year = returnDf[returnDf['Date'].dt.year == year]
    returnDf_year['Return'] = returnDf_year['Return'].astype(float)
    print(str(year)+"Returns",np.prod(1 + returnDf_year['Return']))

In [ ]:
np.mean(returnDf["Return"]) * 252

In [ ]:
print("TimeInput", TimeInput,"\n")
print("NumStocks", NumStocks,"\n")
print("holdTime", holdTime,"\n")
print("stopLoss", stopLoss,"\n")

print("CumulativeReturn", CumulativeReturn,"\n")
print("SharpeRatio", SharpeRatio,"\n")
print("Profit_Loss_Ratio", pLratio,"\n")
print("avg_prof",avg_prof,"\n")
print("avg_loss", avg_loss,"\n")

print("Max_Draw_Down",max_drawdown,"\n")
print("sortino_ratio",sortino_ratio,"\n")

print("Anualized Std",np.std(returnDf["Return"])*np.sqrt(252))



In [ ]:
def objective(trial):
    global experiment_counter
    expirement_name = f"Expirement {experiment_counter}"
    
    with mlflow.start_run(run_name=expirement_name) as run:
        #Setup Paramter Variation
        TimeInput = trial.suggest_int('TimeInput', 2, 7)
        NumStocks=trial.suggest_int('NumStocks', 1, 5)
        holdTime=0
        stopLoss=trial.suggest_float('stopLoss',-0.05, -0.01)

        #Date Range
        DateRange=['2015-01-02','2024-11-07']
        DateRange=[cal.index(DateRange[0]),cal.index(DateRange[1])]
        TotalReturnList = list()

        

        #Run Algorithim
        for day in cal[DateRange[0]:DateRange[1]]:
            dailyReturn = pick_trade(RunDate=day,NumStocks=NumStocks,TimeInput=TimeInput,stopLoss=stopLoss,holdTime=holdTime,debug=False)
            TotalReturnList.append(dailyReturn)
        returnDf = pd.DataFrame({'Date': pd.to_datetime(cal[DateRange[0]:DateRange[1]]),'Return':TotalReturnList})
        returnDf['Date'] = pd.to_datetime(returnDf['Date'])

        #Compound Retruns
        M = 1
        returnVal=[]
        for i in returnDf["Return"]:
            M = M*(1+i)
            returnVal.append(M)
        returnDf=pd.concat([returnDf,pd.DataFrame({"ReturnVals":returnVal})],axis=1)

        #Calculate Metrics
        SharpeRatio =(np.mean(returnDf["Return"]) * 252 - 0.05) / (np.std(returnDf["Return"]) * np.sqrt(252))
        CumulativeReturn = returnDf.loc[len(returnDf)-1,"ReturnVals"]
        max_drawdown = calculate_mdd(returnDf["Return"])
        sortino_ratio = calculate_sortino_ratio(returnDf["Return"])

        
        pLratio = len(returnDf["Return"][returnDf["Return"]>0])/ len(returnDf["Return"])

        avg_loss= np.mean(returnDf["Return"][returnDf["Return"]<0])
        avg_prof= np.mean(returnDf["Return"][returnDf["Return"]>0])
        


        #Saving A plot
        plt.figure(figsize=(15, 6))  # Set the figure size
        plt.plot(returnDf['Date'], returnDf['ReturnVals'],  linestyle='-', color='blue', label="Return Vals")
        plt.title("Return Values Over Time", fontsize=16)
        plt.xlabel("Date", fontsize=12)
        plt.ylabel("Return Vals", fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.legend(fontsize=12)
        plt.savefig("returns.png", dpi=300, bbox_inches='tight')
        plt.close()
        #LOG Paramters, Metrics and Artifacts

        mlflow.log_param("TimeInput", trial.params['TimeInput'])
        mlflow.log_param("NumStocks", trial.params['NumStocks'])
        #mlflow.log_param("holdTime", trial.params['holdTime'])
        mlflow.log_param("stopLoss", trial.params['stopLoss'])

        mlflow.log_metric("CumulativeReturn", CumulativeReturn)
        mlflow.log_metric("SharpeRatio", SharpeRatio)
        mlflow.log_metric("Profit_Loss_Ratio", pLratio)
        mlflow.log_metric("avg_prof",avg_prof)
        mlflow.log_metric("avg_loss", avg_loss)
        
        mlflow.log_metric("Max_Draw_Down",max_drawdown)
        mlflow.log_metric("sortino_ratio",sortino_ratio)

        
        
        yearly_plots(returnDf)
        
        mlflow.log_artifact("returns.png", artifact_path="Visulizations")
        
        for year in range(2015,2025):
            returnDf_year = returnDf[returnDf['Date'].dt.year == year]
            returnDf_year['Return'] = returnDf_year['Return'].astype(float)
            mlflow.log_metric(str(year)+"Returns",np.prod(1 + returnDf_year['Return']))
            mlflow.log_artifact(f"returns_{year}.png", artifact_path="Visulizations")

            
        experiment_counter += 1
        
        return CumulativeReturn, SharpeRatio, sortino_ratio, max_drawdown